<center><img src="img/logo_hse_black.jpg"></center>

<h1><center>Data Analysis</center></h1>
<h2><center>Seminar Introduction to Natural Language Processing<sup><a href="#fn1" id="ref1">1</a></sup></center></h2>

### Sentiment Analysis in Russian (from https://www.kaggle.com/c/sentiment-analysis-in-russian/data)

#### Load data

In [1]:
from tqdm import tqdm_notebook

In [2]:
import json

with open('Data/train.json') as json_file:
    data = json.load(json_file)

#### Tokenize and clean data

In [3]:
import nltk
nltk.download('stopwords')
import string
word_tokenizer = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/r.britkov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
def process_data(data):
    texts = []
    targets = []

    for item in data:
        if item['sentiment'] == 'negative':
            targets.append(0)
        else:
            targets.append(1)
        
        tokens = word_tokenizer.tokenize(item['text'].lower())
        
        #delete punct and stop words
        tokens = [word for word in tokens if (word not in string.punctuation and word not in stop_words)]
        
        texts.append(tokens)
    
    return texts, targets

In [5]:
texts, y = process_data(data)

#### Normalize words. 2 Ways

#### 1)Stemming

In [6]:
from nltk.stem.snowball import SnowballStemmer 

stemmer = SnowballStemmer("russian")

In [7]:
texts_copy = texts[:100]
for i in tqdm_notebook(range(len(texts_copy))):
    texts_copy[i] = ' '.join(list(map(stemmer.stem, texts_copy[i])))

In [8]:
print(texts_copy[0])

досудебн расследован факт покупк енпф пакет облигац то бузгул аурум начат инициатив национальн банк рк сообщ директор департамент защит прав потребител финансов услуг нацбанк казахста александр терент основан досудебн расследован стал обращен национальн банк письм 25 ноябр 2016 год обращен национальн банк правоохранительн орга нам эт сделк показа сомнительн недостаточн корректн поэт нацбанк 25 ноябр 2016 год обрат правоохранительн орга эт мог озвуч сегодн идет следств провод проверк ", – сказа терент 28 декабр нацбанк заяв знают стал основан проверк енпф 23 декабр факт проведен проверк а един накопительн пенсион фонд подтверд пресс служб национальн банк сообщ проверк провод операц совершен а енпф отношен инвестирован собствен актив такж финрегулятор сообща сделк енпф сумм пят млрд завед уголовн дел нацбанк заверя все происходя затрагива пенсион накоплен казахстанц нашл ошибк текст выдел мыш нажм ctrl enter


Advantages: fast

Disadvantages: not very intellectual normalization

Alternative: lemmatization (but this method slomly than stemming)

#### 2)Lemmatize

In [9]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [10]:
texts_copy = texts[:100]
for i in tqdm_notebook(range(len(texts_copy))):
    texts_copy[i] = ' '.join([morph.parse(word)[0].normal_form for word in texts_copy[i]])

In [11]:
print(texts_copy[0])

досудебный расследование факт покупка енпф пакет облигация тоо бузгул аурум начать инициатива национальный банка рк сообщить директор департамент защита право потребитель финансовый услуга нацбанк казахстан александр терентьев основание досудебный расследование стать обращение национальный банка письмо 25 ноябрь 2016 год обращение национальный банка правоохранительный орган мы этот сделка показаться сомнительный недостаточно корректный поэтому нацбанк 25 ноябрь 2016 год обратиться правоохранительный орган это мочь озвучить сегодня идти следствие проводиться проверка ", – сказать терентьев 28 декабрь нацбанк заявить знать стать основание проверка енпф 23 декабрь факт проведение проверка ао единый накопительный пенсионный фонд подтвердиться пресс служба национальный банка сообщить проверка проводить операция совершить ао енпф отношение инвестирование собственный актив также финрегулятор сообщать сделка енпф сумма пять миллиард завести уголовный дело нацбанк заверять весь происходить затр

We will use stemmer because it is enough for our task

In [12]:
for i in tqdm_notebook(range(len(texts))):
    texts[i] = ' '.join(list(map(stemmer.stem, texts[i])))

In [13]:
#train test_split
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_y, test_y = train_test_split(texts, y, test_size=0.33, random_state=42, stratify = y)

### Recap TF-IDF

TF-IDF: method to measure importance of word in text

$$
TF-IDF(t,d) = tf(t,d) \cdot idf(t, D)
$$

where $tf(t,d)$ (in simple case) raw count of a term t in a document d.
$$
idf(t, D) = \log \frac{N}{N(t)}
$$
where $N$ - total number of documents, $N(t)$ - total number of documents with term t

In [14]:
#calc tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

In [15]:
#Example on small dataset

In [16]:
texts_head = train_texts[:100]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(texts_head)
vectorizer.get_feature_names()

['00',
 '000',
 '006',
 '01',
 '013',
 '018',
 '02',
 '025',
 '03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '080',
 '089',
 '09',
 '10',
 '100',
 '1000',
 '100er',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '1089',
 '11',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '12',
 '120',
 '1200',
 '121',
 '122',
 '125',
 '127',
 '13',
 '130',
 '137',
 '1380',
 '139',
 '14',
 '140',
 '141',
 '1418',
 '142',
 '1446',
 '146',
 '148',
 '1497',
 '15',
 '150',
 '1500',
 '1503',
 '153',
 '1560',
 '159',
 '16',
 '160',
 '161',
 '162',
 '1640',
 '165',
 '168',
 '16г',
 '17',
 '170',
 '1719',
 '1725',
 '173',
 '1739',
 '1759',
 '176',
 '177',
 '1775',
 '178',
 '179',
 '18',
 '180',
 '1800',
 '181',
 '1831',
 '185',
 '187',
 '188',
 '189',
 '1899',
 '19',
 '190',
 '1900',
 '1902',
 '1913',
 '1932',
 '1934',
 '1936',
 '1938',
 '1939',
 '1940',
 '1943',
 '1946',
 '1947',
 '1949',
 '195',
 '1950',
 '1952',
 '1953',
 '1954',
 '1955',
 '1956',
 '1957',
 '1958',
 '1959',
 '1960',
 '1962',
 '1964'

How you see many useless words (numbers, english words). Let's use filtering

1)Filter by min_df. When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold

In [17]:
vectorizer = TfidfVectorizer(min_df = 0.2) #filter all word which appear less than 20% of documents
X = vectorizer.fit_transform(texts_head)
vectorizer.get_feature_names()

['10',
 '12',
 '14',
 '20',
 '2015',
 '2016',
 '2017',
 '30',
 'kz',
 'алмат',
 'аста',
 'банк',
 'больш',
 'будут',
 'вид',
 'возможн',
 'вопрос',
 'врем',
 'глав',
 'год',
 'государств',
 'государствен',
 'дан',
 'декабр',
 'дел',
 'ден',
 'деятельн',
 'директор',
 'должн',
 'друг',
 'един',
 'занима',
 'информац',
 'итог',
 'казахста',
 'казахстан',
 'казахстанск',
 'компан',
 'котор',
 'кром',
 'крупн',
 'лет',
 'лиц',
 'международн',
 'мер',
 'мест',
 'месяц',
 'министерств',
 'министр',
 'млн',
 'млрд',
 'направлен',
 'населен',
 'наход',
 'национальн',
 'нача',
 'наш',
 'необходим',
 'нов',
 'обеспечен',
 'област',
 'общ',
 'обь',
 'одн',
 'отмет',
 'перв',
 'переда',
 'период',
 'планир',
 'получ',
 'порядк',
 'правительств',
 'предприят',
 'председател',
 'представител',
 'пресс',
 'провод',
 'программ',
 'проект',
 'производств',
 'прошл',
 'работ',
 'работа',
 'развит',
 'размер',
 'рамк',
 'ран',
 'реализац',
 'результат',
 'республик',
 'решен',
 'рк',
 'рынк',
 'сайт',
 '

2)Filter by max_df. When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words)

In [18]:
vectorizer = TfidfVectorizer(min_df = 0.2, max_df = 0.95) #filter all word which appear less than 20% and great than 95% of documents
X = vectorizer.fit_transform(texts_head)
vectorizer.get_feature_names()

['10',
 '12',
 '14',
 '20',
 '2015',
 '2016',
 '2017',
 '30',
 'kz',
 'алмат',
 'аста',
 'банк',
 'больш',
 'будут',
 'вид',
 'возможн',
 'вопрос',
 'врем',
 'глав',
 'год',
 'государств',
 'государствен',
 'дан',
 'декабр',
 'дел',
 'ден',
 'деятельн',
 'директор',
 'должн',
 'друг',
 'един',
 'занима',
 'информац',
 'итог',
 'казахста',
 'казахстан',
 'казахстанск',
 'компан',
 'котор',
 'кром',
 'крупн',
 'лет',
 'лиц',
 'международн',
 'мер',
 'мест',
 'месяц',
 'министерств',
 'министр',
 'млн',
 'млрд',
 'направлен',
 'населен',
 'наход',
 'национальн',
 'нача',
 'наш',
 'необходим',
 'нов',
 'обеспечен',
 'област',
 'общ',
 'обь',
 'одн',
 'отмет',
 'перв',
 'переда',
 'период',
 'планир',
 'получ',
 'порядк',
 'правительств',
 'предприят',
 'председател',
 'представител',
 'пресс',
 'провод',
 'программ',
 'проект',
 'производств',
 'прошл',
 'работ',
 'работа',
 'развит',
 'размер',
 'рамк',
 'ран',
 'реализац',
 'результат',
 'республик',
 'решен',
 'рк',
 'рынк',
 'сайт',
 '

3)max_features build a vocabulary that only consider the top max_features ordered by term frequency across the corpus

In [19]:
vectorizer = TfidfVectorizer(max_features = 25) #only top-25 words ordered by tf
X = vectorizer.fit_transform(texts_head)
vectorizer.get_feature_names()

['2016',
 '2017',
 'вопрос',
 'год',
 'казахста',
 'казахстанск',
 'компан',
 'котор',
 'лет',
 'назад',
 'национальн',
 'необходим',
 'нов',
 'област',
 'работ',
 'развит',
 'район',
 'республик',
 'рк',
 'стран',
 'суд',
 'такж',
 'тенг',
 'трудов',
 'эт']

If you want to get the column number associated with the token, you can use mapping from vectorizer.vocabulary_

In [20]:
vectorizer.vocabulary_

{'котор': 7,
 'лет': 8,
 'год': 3,
 'такж': 21,
 'эт': 24,
 'тенг': 22,
 'казахстанск': 5,
 'компан': 6,
 '2016': 0,
 'вопрос': 2,
 'рк': 18,
 'област': 13,
 'республик': 17,
 'казахста': 4,
 'развит': 15,
 'работ': 14,
 'стран': 19,
 'нов': 12,
 '2017': 1,
 'национальн': 10,
 'необходим': 11,
 'район': 16,
 'суд': 20,
 'назад': 9,
 'трудов': 23}

In our case we will use simple max_features filtering

In [21]:
vectorizer = TfidfVectorizer(max_features = 40000)
X = vectorizer.fit_transform(train_texts)

X is matrix with shape = (number of sentences $\times$ number of words). Each coordinate is tf-idf for corresponding words. If in coordinate will be count of word in sentence this is call "Bag-of-words"

In [22]:
X.shape

(5536, 40000)

Now let's fit log-reg model, but 40000 is too large for model. Use LSA to reduce dimension. In sklearn LCA is TruncatedSVD

In [23]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components = 1000)

In [24]:
%%time
X_small = svd.fit_transform(X)

CPU times: user 41 s, sys: 3.63 s, total: 44.6 s
Wall time: 33.3 s


In [25]:
X_small.shape

(5536, 1000)

In [26]:
from sklearn.linear_model import LogisticRegression

In [27]:
model = LogisticRegression()
model.fit(X_small, train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [28]:
predict = model.predict(X_small)
proba = model.predict_proba(X_small)

In [29]:
from sklearn.metrics import accuracy_score, roc_auc_score
print("ACCURACY = {}".format(accuracy_score(train_y, predict)))
print("ROC-AUC = {}".format(roc_auc_score(train_y, proba[:, 1])))

ACCURACY = 0.8876445086705202
ROC-AUC = 0.9494297265485065


### Evaluate on test data

In [30]:
test_X = vectorizer.transform(test_texts).toarray()
test_X = svd.transform(test_X)

In [31]:
predict = model.predict(test_X)
proba = model.predict_proba(test_X)
print("ACCURACY = {}".format(accuracy_score(test_y, predict)))
print("ROC-AUC = {}".format(roc_auc_score(test_y, proba[:, 1])))

ACCURACY = 0.8720205353868721
ROC-AUC = 0.9200978856475028


## Kaggle

You can use any models and algorithms that were told at a lecture or a seminar.

#### Example

### Load data

In [32]:
import pandas as pd
import numpy as np

In [33]:
data = pd.read_csv('./data/train.tsv', sep = '\t')

In [34]:
data.head()

,Unnamed: 0,category_id,city,date_created,delivery_available,desc_text,img_num,lat,long,name_text,owner_id,payment_available,price,product_id,product_type,properties,region,sold_mode,subcategory_id,sold_fast
0,1,4,Краснодар,2018-10-08,False,"Продаю стол раскладной, деревянный, советский ...",3,45.0686,38.9518,Стол,4ce583fe8231a0cc4a3c7d241c7d0289,True,500.0,8cb80c05c65c210275f5500779d6b593,1,"[{'slug_id': 'stoly_stulya_tip', 'slug_name': ...",Краснодарский край,1,410,1
1,2,4,Тюмень,2018-06-18,False,"Тарелки глубокие 6 шт. Блюдца, чашки по 6 шт. ...",2,57.1840,65.5674,Посуда,e58be2c8f143c17246dc2243b5d3b98f,False,300.0,3b7a9f8b27a53b63525f95bc8070abb2,1,"[{'slug_id': 'dom_dacha_posuda_tip', 'slug_nam...",Тюменская область,1,405,0
2,4,9,Омск,2018-07-31,True,"Новый,с этикеткой. Размер L. Не подошёл по раз...",1,54.9889,73.4312,Костюм,51b408796027214232532b7e478e2159,True,1100.0,c97dd9c5a3e938c52cf5d7822bc0eb7b,1,[{'slug_id': 'zhenskaya_odezhda_pidzhaki_kosty...,Омская область,1,908,0
3,6,3,Санкт-Петербург,2018-04-17,False,"Складывается тростью, все колеса вниз. Сплошна...",4,59.9590,30.4877,Коляска,6544b83acbbf04439a7ba983093cafb4,True,5000.0,3e5d0286b25fd7f62f88bc436a59ae4e,1,"[{'slug_id': 'waggon_type', 'slug_name': 'Тип'...",Ленинградская область,1,312,0
4,10,5,Москва,2018-02-09,False,"Неразлучники, птичкам по 1,5 года. Продаю с бо...",2,55.6473,37.4118,Волнистые попугаи,ea575e28daf1f47bfce63015cd3ce5cf,True,2000.0,57b4a8679d0d3eb1e31367b57221098f,1,[],Московская область,1,504,0


In [35]:
from collections import defaultdict, Counter
def target_encoding(features, targets):
    values = defaultdict(int)
    counts = Counter()
    for val, target in zip(features, targets):
        values[val] += target
        counts[val] += 1
    
    mean_values = dict()
    for val in values:
        mean_values[val] = values[val] / counts[val]
    
    return mean_values

In [36]:
def preprocess_cat_features(data, cat_features, target):
    cat_features_dict = dict()
    for feature in cat_features:
        cat_features_dict[feature] = target_encoding(data[feature].values, data[target].values)
    return cat_features_dict

In [37]:
#use real features
X = data[['lat', 'long', 'price']].values
y = data['sold_fast']

In [38]:
cat_features = ['category_id', 'city', 'delivery_available', 'img_num', 'region']
cat_features_dict = preprocess_cat_features(data, cat_features, 'sold_fast')

In [39]:
for feature in cat_features:
    res = [0] * len(data)
    for i, val in enumerate(data[feature].values):
        res[i] = cat_features_dict[feature][val]
    X = np.c_[X, np.array(res)]

###  We dont'use text data, because don't have enough time on seminar for preprocessing

Always shuffle your data and don't forget fix random_seed and random_state

In [40]:
from sklearn.utils import shuffle

In [41]:
X, y = shuffle(X, y, random_state = 42)

In [42]:
model = LogisticRegression()
model.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

### Evaluate on test_data and save file for submit

In [43]:
data = pd.read_csv('./data/test_nolabel.tsv', sep = '\t')

In [44]:
X = data[['lat', 'long', 'price']].values
for feature in cat_features:
    res = [0] * len(data)
    for i, val in enumerate(data[feature].values):
        res[i] = cat_features_dict[feature].get(val, 0)
    X = np.c_[X, np.array(res)]

In [45]:
proba = model.predict_proba(X)

In [47]:
product_id = data['product_id'].values
data = pd.DataFrame.from_dict({'product_id' : product_id, 'score' : proba[:, 1]})
data.to_csv('./to_submit', sep = ',', index = False)